# 多输入多输出通道
实现一下多输入通道互相关运算

In [20]:
import torch
from d2l import torch as d2l

# 对X,K的每一个channel做卷积
def corr2d_multi_in(X,K):
    return sum(d2l.corr2d(x,k) for x,k in zip(X,K))

验证互相关运算的输出
<br/> multi-input  --->  one output

In [21]:
# 2 channel
# 3x3
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
                  [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
# 2 channel
# 2x2 kernel
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

# output  ----> 2x2 单输出
X.shape
K.shape
corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

<br/> multi-input  --->  multi output

In [22]:
def corr2d_multi_in_out(X, K):
    # axis = 0 堆
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

# 3 channel 的 output 3 x 2x 2x 2
K = torch.stack((K, K + 1, K + 2), 0)
K.shape



torch.Size([3, 2, 2, 2])

In [23]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

1x1卷积  --- 等价一个全连接层

In [26]:
def corr2d_multi_in_out_1x1(X,K):
    c_i,h,w = X.shape # input channel, height,width
    c_o = K.shape[0] # output channel
    # 拉平vec,变成一个全连接层
    X = X.reshape((c_i, h * w))
    #weight
    K = K.reshape((c_o, c_i))
    Y = torch.matmul(K,X)
    #再reshape 回矩阵
    return Y.reshape(c_o,h,w)

# N~(0,1) channel x h x w
X = torch.normal(0, 1, (3, 3, 3))
# output(channel) x input channelx kernel size
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6
